# Data chunking and zarr

## Authors & Contributors
### Authors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)
- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)

### Contributors
- Anne Fouilloux, University of Oslo (Norway), [@annefou](https://github.com/annefou)
- Guillaume Eynard-Bontemps, CNES (France), [@guillaumeeb](https://github.com/guillaumeeb)



<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> <b>Overview</b>
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>Why do chunking matter?</li>
        <li>How can I read datasets by chunks to optimize memory usage?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about chunking</li>
        <li>Learn about zarr </li>
    </ul>
</div>

## Context

When dealing with large data files or collections, it's often impossible to load all the data you want to analyze into a single computer's RAM at once. This is a situation where the Pangeo ecosystem is well-suited. `Xarray` offers the possibility to work lazily on data __chunks__, which means pieces of an entire dataset. By reading a dataset in __chunks__ we can process our data piece by piece on a single computer and even on a distributed computing cluster using Dask (Cloud or HPC for instance).

How we will process these 'chunks' in a parallel environment will be discussed in [dask_introduction](./dask_introduction.ipynb). The concept of __chunk__ will be explained here.

When we process our data piece by piece, it's easier to have our input or ouput data also saved in __chunks__. [Zarr](https://zarr.readthedocs.io/en/stable/) is the reference library in the Pangeo ecosystem to save our `Xarray` multidimentional datasets in __chunks__.

The analysis is very similar to what we have done in previous episodes, however we will use data on a global coverage and not only on a small geographical area (e.g. Lombardia).

### Data

In this episode, we will be using 

## Setup

This episode uses the following main Python packages:

- xarray {cite:ps}`d-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- dask {cite:ps}`d-dask-2016`
- geopandas {cite:ps}`d-geopandas-jordahl2020`
- matplotlib {cite:ps}`d-matplotlib-Hunter2007`

Please install these packages if not already available in your Python environment (see [Setup page](https://pangeo-data.github.io/foss4g-2022/before/setup.html)).

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

## title here

we are ...

In [ ]:
import pystac_client
import geopandas as gpd
from shapely.geometry import mapping
import stackstac
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
aoi = gpd.read_file('../data/catchment_outline.geojson', crs="EPGS:4326")
aoi_geojson = mapping(aoi.iloc[0].geometry)
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
items = catalog.search(
    intersects=aoi_geojson,
    collections=["sentinel-2-l2a"],
    datetime="2019-02-01/2019-06-10"
).item_collection()
sentinel2_l2a = stackstac.stack(items)

In [ ]:
sentinel2_l2a

## What is a __chunk__

If you look carefully to `sentinel2_l2a`,  xarray.DataArray is a `dask.array` with a chunk size of `(1, 1, 1024, 1024)`. The full data would load arrays of dimensions `(101, 32, 20982, 10980)`, 746 592 of the 'chunk', in total 5.42 TiB into the computer's RAM.  

We can see the `dask.array` information by clicking the icon as circled blue in the image below.

![Dask.array](../figures/datasize.png)

By clicking the red circled triangle icon, we can have detailed information on the `xarray.DataArray`, such as Coordinates, Indexes and Attributes.

When you create Xarray object using `stackstac`, we can easily turns STAC collection into a lazy `xarray.DataArray`, in chunk form, so then it is backed by dask.

The size and shape of chunk which we will use defines the parallelisation done by Dask, thus picking a good chunksize will have significant effects on performance.

This is where understanding and using chunking correctly comes into play.

__Chunking__ is splitting a dataset into small pieces. 

Original dataset, in one piece,  
<img src="../figures/notchunked.png" width="200" height="100">

and we split it into several smaller pieces.
<img src="../figures/chunked.png" width="200" height="100">

We split it into pieces so that we can process our data block by block or __chunk__ by __chunk__.

In our case, for the moment, we used stackstac without specifying 'chunk' explicitly. The dataset is composed of 8MiB each, each contains, 1 time step, 1 band, 1024 x 1024 on x and y direction.
<img src="../figures/chunk_original.png" width="200" height="100">

If we have a too small chunk size, we will divide our work flow in too small pieces, which can create too many communications, too many 'distribution' overheads.
If we have a too big chunk size, we may not be able to hold the enough memory and our workflow may die.

The right size of chunk depends on your computation and the machine you use.

Here, 8MiB, is very small compare to usual RAM size available. For example, dask's default array size is 128MiB.

In [ ]:
import dask
dask.config.get('array.chunk-size')

## Modifying chunks

Lets try to modify our chunk size.

To modify chunks on your existing `xarray.DataArray` we can use the `chunk` function.
We know that we only need 3 bands for computing the snow index example, so we select only `green`,`swir16` and `scl` to simplify our example.

We would like to have each time series separated in each chunk, then keep all band informnation on one chunk, and let dask to compute x and y coordinate's chunk size.

In [ ]:
sentinel2_l2a=sentinel2_l2a.sel(
    band=['green','swir16','scl']).chunk(
    chunks={'time': 1, 'band':3, 'x':'auto','y':'auto'})
sentinel2_l2a

If you look into details of any variable in the representation above, you'll see that each x and y coordinate's chunk is bigger, and we have much less chunks (6666 chunks) than the example before. A chunk size of 96MiB is already more manageable than 8MiB small chunk.

In [ ]:
sentinel2_l2a.chunk(chunks = ( 1, -1, 12048,2048))

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Go Further</b>
    <br>
    <br>
    You can try to apply different ways for specifying chunk.
    <ul>
        <li> chunks = -1 -> the entire array will be used as a single chunk
        <li> chunks = {'x':-1, 'y': 1000} -> chunks of entire _x_ dimension, but splitted every 1000 values on _y_ dimension</li>
        <li> chunks = {'x':-1, 'y': 'auto'} -> Xarray relies on Dask to use an ideal size according to the preferred chunk sizes for _y_ dimension</li>
        <li> chunks = { 'x':-1 ,'y':"500MiB" } -> Xarray seeks the size according to a specific memory target expressed in MiB</li>
        <li> chunks = ( 1, 3, 12048,2048) -> Specifying chunk size in the order of dimension. </li>
    </ul>
</div>

## Defining the chunk at the creatioin of Xarray

We can define the chunk size when we create the object.  
This is usually done with Xarray using the `chunks` kwarg when opening a file with `xr.open_dataset` or with `xr.open_mfdataset`, if you create Xarray from your local file.  
In our snow index example, we create Xarray from stackstac. As stackstac's default 'chunksize' definition is 1024 for x and y dimension, we had that chunksize.  We can pass the chunksize option to stdeackstac and make that bigger.


In [ ]:
%%time
sentinel2_l2a = stackstac.stack(items
                                ,assets=['green','swir16','scl']
                               ,chunksize=( 1, 3, 2048,2048)
)
sentinel2_l2a

## So, why chunks?

Chunks are mandatory for accessing files or dataset that are bigger than a single computer's memory. If all the data has to be accessed, it can be done sequentially e.g. chunks are processed one after the othe).

Moreover, chunks allow for distributed processing and so increased speed for your data analysis, as seen in the next episode.

### Chunks and files

Xarray chunking possibilities also relies on the underlying input or output file format used. Most modern file format allows to store a dataset or a single file using chunks. NetCDF4 uses chunks when storing a file on the disk through the use of HDF5. Any read of data in a NetCDF4 file will lead to the load of at least one chunk of this file. So when reading one of its chunk as defined in `open_dataset` call, Xarray will take advantage of native file chunking and won't have to read the entire file too.


Yet, it is really important to note that __Xarray chunks and file chunks are not necessarily the same__. It is however a really good idea to configure Xarray chunks so that they align well on input file format chunks (so ideally, Xarray chunks should contain one or several input file chunks).

## Zarr storage format

This brings to our next subjects [Zarr](https://zarr.readthedocs.io/en/stable/) and [Kerchunk](https://fsspec.github.io/kerchunk/).

If we can have our original dataset already 'chunked' and accessed in an optimized way according to it's actual byte storage on disk, we won't need to load entire dataset every time, and our data anlayzis, even working on the entire dataset, will be greatly optimized.

Let's convert our intermediate data into Zarr format so that we can learn what it is. We can keep the data as in DataArray or convert that into DataSet before storing them.

We start again from loading data using stackstac, but this time we go to next step, clipping the data and computation of snow index, and lets try to save those intermediate result in a zarr file.  


In [ ]:
# Data Manipulation and Analysis Libraries
import pandas as pd  
import numpy as np 

# Geospatial Data Handling Libraries
import geopandas as gpd 
from shapely.geometry import mapping  
import pyproj

# Multidimensional and Satellite Data Libraries
import xarray as xr 
import rioxarray as rio
import stackstac

# Data Visualization Libraries
import holoviews as hv
import hvplot.xarray
import hvplot.pandas

# Data parallelization and distributed computing libraries
import dask
from dask.distributed import Client, progress, LocalCluster

# STAC Catalogue Libraries
import pystac_client

import warnings
warnings.filterwarnings("ignore")

In [ ]:
cluster = LocalCluster()
client = Client(cluster)
client

## Load data using stackstac (with specific chunk) 

In [ ]:
%%time
aoi = gpd.read_file('../data/catchment_outline.geojson', crs="EPGS:4326")
aoi_geojson = mapping(aoi.iloc[0].geometry)
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
items = catalog.search(
    intersects=aoi_geojson,
    collections=["sentinel-2-l2a"],
    datetime="2019-02-01/2019-06-10"
).item_collection()
ds = stackstac.stack(items
                                ,assets=['green','swir16','scl']
                               ,chunksize=( 1, 3, 1024,1024)
)
#ds

## Coomputing Snow index


In [ ]:
green = ds.sel(band='green')
swir = ds.sel(band='swir16')
scl = ds.sel(band='scl')
ndsi = (green - swir) / (green + swir)
snow = xr.where((ndsi > 0.42) & ~np.isnan(ndsi), 1, ndsi)
snowmap = xr.where((snow <= 0.42) & ~np.isnan(snow), 0, snow)
mask = np.logical_not(scl.isin([8, 9, 3])) 
snow_cloud = xr.where(mask, snowmap, 2)
#snow_cloud

## Clip the data

In [ ]:
aoi_utm32 = aoi.to_crs(epsg=32632)
geom_utm32 = aoi_utm32.iloc[0]['geometry']
snow_cloud.rio.write_crs("EPSG:32632", inplace=True)
snow_cloud.rio.set_nodata(np.nan, inplace=True)
snow_cloud = snow_cloud.rio.clip([geom_utm32])
#snow_cloud

## Lets save the intermediate result of a few days in a zarr format

In [ ]:
test=snow_cloud.isel(time=slice(0,3))
test

## Before saving, we clean the chunk size, clean attribute and save the data in Xarray DataSet format


In [ ]:
test=test.chunk(chunks = {'x':'auto', 'y': 'auto'}).to_dataset(name='data')
test


In [ ]:
%%time

def remove_attrs(obj, to_remove):
    new = obj.copy()
    new.attrs = {k: v for k, v in obj.attrs.items() if k not in to_remove}

    return new

def encode(obj):
    object_coords = [name for name, coord in obj.coords.items() if coord.dtype.kind == "O"]
    return obj.drop_vars(object_coords).pipe(remove_attrs, ["spec", "transform"])


test.pipe(encode).to_zarr('test.zarr',mode='w')

## Load the temporaly data saved as zarr back to Xarray

In [ ]:
snow_cloud=xr.open_zarr('test.zarr').data

## Group by to a day, and plot the results

In [ ]:
clipped_date = snow_cloud.groupby(snow_cloud.time.dt.floor('D')).max(skipna=True)
clipped_date = clipped_date.rename({'floor': 'date'})

In [ ]:
clipped_date.hvplot.image(
    x='x',
    y='y',
    groupby='date',
    crs=pyproj.CRS.from_epsg(32632),
    cmap='Pastel2',
    clim=(-1, 2),
    frame_width=500,
    frame_height=500,
    title='Snowmap',
    geo=True, tiles='OSM')

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Exercise</b>
    <br>
    <ul>
        <li>What about saving the data in Netcdf format? `ls -la test.zarr` and  `ls -la test.zarr/nobs `</li>
        <li>You can try to explore the zarr file you just created using `ls -la test.zarr` and  `ls -la test.zarr/nobs `</li>
        <li>You can explore zarr metadata file by `cat test.zarr/.zmetadata` </li>
        <li>Did you find the __chunks__ we defined previously in your zarr file? </li>
    </ul>
</div>

In [ ]:
xr.open_zarr('test.zarr').to_netcdf('test.nc')

In [ ]:
!du -sh test.zarr/ test.nc

In [ ]:
!ls  -la test.zarr/data

In [ ]:
!cat test.zarr/.zmetadata | head -n 30

Zarr format main characteristics are the following:

- Every chunk of a Zarr dataset is stored as a single file (see x.y files in `ls -al test.zarr/data`)
- Each Data array in a Zarr dataset has a two unique files containing metadata:
  - .zattrs for dataset or dataarray general metadatas
  - .zarray indicating how the dataarray is chunked, and where to find them on disk or other storage.
  
Zarr can be considered as an Analysis Ready, cloud optimized data (ARCO) file format, discussed in [data_discovery](./data_discovery.ipynb) section.

## Conclusion

Understanding chunking is key to optimize your data analysis when dealing with big datasets. In this episode we learned how to optimize the data access time and memory resources by exploiting native file chunks from netCDF4 data files and instructing Xarray to access data per chunk. However, computations on big datasets can be very slow on a single computer, and to optimize its time we may need to parallelize your computations. This is what you will learn in the next episode with Dask.

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Chunking </li>
        <li>zarr </li>
        <li>kerchunk</li>
    </ul>
</div>

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "chunking" and topic % "package"
:keyprefix: d-
```